In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import gzip
import io
import numpy as np
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:
        try:
            result = {}
            result['url'] = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
            lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
            if lastmod is not None:
                result['lastmod'] = lastmod.text

            #title = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:title').text
            #keywords = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:keywords').text
            #news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))
            news_list.append(result)
        except:
            continue

    return news_list

In [3]:
sitemap_urls = parse_sitemap(io.StringIO(req.get('https://meduza.io/sitemap/full.xml').content.decode()))
sitemap_urls

[{'url': 'https://meduza.io/sitemap/part_1.xml'},
 {'url': 'https://meduza.io/sitemap/part_2.xml'},
 {'url': 'https://meduza.io/sitemap/part_3.xml'},
 {'url': 'https://meduza.io/sitemap/part_4.xml'},
 {'url': 'https://meduza.io/sitemap/part_5.xml'},
 {'url': 'https://meduza.io/sitemap/part_6.xml'},
 {'url': 'https://meduza.io/sitemap/part_7.xml'},
 {'url': 'https://meduza.io/sitemap/part_8.xml'},
 {'url': 'https://meduza.io/sitemap/part_static.xml'}]

In [4]:
news_db = []

for fn in tqdm.tqdm_notebook(sitemap_urls):
    s1 = time.time()

    try:
        text = io.StringIO(req.get(fn['url']).content.decode())
        news = parse_sitemap(text)
    except Exception as e:
        print(e)
        continue
        
    news_db += news
    s2 = time.time()
    time.sleep(max(1 - s2 + s1, 0))

<ipython-input-4-5701dbbfbec7>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fn in tqdm.tqdm_notebook(sitemap_urls):


In [5]:
df = pd.DataFrame(news_db)
df['lastmod'] = pd.to_datetime(df['lastmod'], format='%Y-%m-%dT%H:%M:%S', utc=True)
df.head()

,url,lastmod
0,https://meduza.io/en/news/2015/09/28/russia-na...,2016-12-10 05:57:25+00:00
1,https://meduza.io/news/2015/11/19/v-kieve-sove...,2016-12-10 05:57:25+00:00
2,https://meduza.io/shapito/2016/02/17/avstraliy...,2016-12-10 05:57:26+00:00
3,https://meduza.io/news/2014/10/30/po-delu-o-kr...,2016-12-10 05:57:26+00:00
4,https://meduza.io/feature/2015/12/01/rebyata-b...,2016-12-10 05:57:31+00:00


In [6]:
df[(df.lastmod >= '2019-11-01')].to_json('news_meduza.json', date_unit='s', date_format='iso')
df[(df.lastmod >= '2019-11-01')].shape
#df[(df.lastmod >= '2018-01-01') & (df.lastmod < '2019-07-01')].shape

(0, 2)

# Parsing

In [2]:
df = pd.read_json('db/news_meduza_part.json')

In [5]:
def parse_news(text):
    soup = BeautifulSoup(text, features='lxml')
    news_text_list = soup.find(name="div", attrs={'class': "GeneralMaterial-article"})

    if news_text_list:
        news_text = ''
        news_elems = news_text_list.find_all(name='p', attrs={'class': ['SimpleBlock-lead', 'SimpleBlock-p']})
        
        for news_elem in news_elems:
            news_text += news_elem.get_text(" ", strip=True)
        news_text = re.sub(r'[\n\r]+', ' ', news_text).replace(u'\xa0', u' ')
    else:
        news_text = ''

    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+', ' ', description_elem.get('content')).replace(u'\xa0', u' ') if description_elem else ''
    
    tags_elem = soup.find(name="meta", attrs={'name': "keywords"})
    tags = re.sub(r'[\n\r]+', ' ', tags_elem.get('content')).replace(u'\xa0', u' ') if tags_elem else ''
    return {"news_text": news_text, "tags": tags, "description": description}

In [6]:
parse_news(req.get('https://meduza.io/feature/2019/06/04/chernobyl-5-seriya-kakova-tsena-lzhi').text)

{'news_text': '4 июня вышел заключительный эпизод сериала «Чернобыль». В финале истории зрители узнают о настоящих причинах взрыва и о том, почему Валерий Легасов совершил самоубийство. «Медуза» рассказывает, что произошло в последней серии и какое самое большое художественное допущение сделали авторы.Серия начинается за 12 часов до взрыва — днем 25 апреля 1986 года Припять живет как обычно: вокруг цветущих клумб бегают дети, взрослые улыбаются друг другу, Людмила Игнатенко выбирает пуговицы в магазине тканей, а ее муж нянчится с соседским ребенком. В это время Анатолий Дятлов идет на станцию — на этот день назначены испытания безопасности реактора, и если все пройдет успешно, повысят всех: и директора станции Виктора Брюханова, и главного инженера станции Николая Фомина, и самого Дятлова. Для испытания нужно понизить мощность — и в эти часы реактор уже работает вполсилы. Но провести испытания днем не получится: Брюханову позвонили из Киева и попросили подождать 10 часов. Дятлов говори

In [10]:
for i, item in enumerate(df.iterrows()):

    if not df.loc[item[0], 'downloaded']:
        
        try:
            # Processing row from db
            url = item[1]['url']
            lastmod = item[1]['lastmod']
            csv_row = dict(zip(['news_id', 'url', 'lastmod'], [item[0], url, lastmod]))

            # Downloading file
            start = time.time()
            resp = req.get(url)
            if resp.status_code == 200:
                csv_row['news_text'], csv_row['tags'], csv_row['description'] = parse_rbk_news(resp.text)

            # Saving row to csv file
            with open('db/news_rbk_part.csv', 'a') as csv_file:
                writer = csv.DictWriter(csv_file, csv_row.keys())
                writer.writerow(csv_row)

            # News is written, logging it to DF
            #df.loc[item[0], 'downloaded'] = True
            
            # Logging to console
            if i % 1000 == 0 and i > 0:
                #df.to_json('db/news_rbk_part.json')
                pass
                
            if i % 100 == 0 and i > 0:
                datetime = time.strftime('%Y-%m-%d %H:%M')
                print(datetime, ':', i, 'news downloaded,', int((df.shape[0]-i) * 1.1 / 60 / 60), 'hours remaining')

            # Calculating wait time (must be 1 sec minimum)
            end = time.time()
            wait = max(0, 1.1 - (end - start))
            time.sleep(wait)
            break
        except Exception as e:
            #df.to_json('db/news_rbk_part.json')
            print(e)
            break